# **Clone Repo and install dependencies**

In [ ]:
!git clone https://github.com/ultralytics/yolov5
!pip install -qr yolov5/requirements.txt
%cd yolov5

import torch
from IPython.display import Image, clear_output
clear_output()

# Download custom dataset

In [ ]:
#dataset custom facut de mine, l-am pus in google drive
!gdown https://drive.google.com/uc?id=1ezuBbiAjEUJwUmTy5rUDSULfwSItQDDV&export=download
!unzip /content/Fire_Detector.zip

# **Use a custom model configuration**

In [ ]:
#customize iPython writefile so we can write variables
from IPython.core.magic import register_line_cell_magic

@register_line_cell_magic
def writetemplate(line, cell):
    with open(line, 'w') as f:
        f.write(cell.format(**globals()))

In [ ]:
%%writetemplate /content/yolov5/models/custom_yolov5s.yaml

# parameters
nc: 2  # number of classes
depth_multiple: 0.33  # model depth multiple
width_multiple: 0.50  # layer channel multiple

# anchors
anchors:
  - [10,13, 16,30, 33,23]  # P3/8
  - [30,61, 62,45, 59,119]  # P4/16
  - [116,90, 156,198, 373,326]  # P5/32

# YOLOv5 backbone
backbone:
  # [from, number, module, args]
  [[-1, 1, Focus, [64, 3]],  # 0-P1/2
   [-1, 1, Conv, [128, 3, 2]],  # 1-P2/4
   [-1, 3, BottleneckCSP, [128]],
   [-1, 1, Conv, [256, 3, 2]],  # 3-P3/8
   [-1, 9, BottleneckCSP, [256]],
   [-1, 1, Conv, [512, 3, 2]],  # 5-P4/16
   [-1, 9, BottleneckCSP, [512]],
   [-1, 1, Conv, [1024, 3, 2]],  # 7-P5/32
   [-1, 1, SPP, [1024, [5, 9, 13]]],
   [-1, 3, BottleneckCSP, [1024, False]],  # 9
  ]

# YOLOv5 head
head:
  [[-1, 1, Conv, [512, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 6], 1, Concat, [1]],  # cat backbone P4
   [-1, 3, BottleneckCSP, [512, False]],  # 13

   [-1, 1, Conv, [256, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 4], 1, Concat, [1]],  # cat backbone P3
   [-1, 3, BottleneckCSP, [256, False]],  # 17 (P3/8-small)

   [-1, 1, Conv, [256, 3, 2]],
   [[-1, 14], 1, Concat, [1]],  # cat head P4
   [-1, 3, BottleneckCSP, [512, False]],  # 20 (P4/16-medium)

   [-1, 1, Conv, [512, 3, 2]],
   [[-1, 10], 1, Concat, [1]],  # cat head P5
   [-1, 3, BottleneckCSP, [1024, False]],  # 23 (P5/32-large)

   [[17, 20, 23], 1, Detect, [nc, anchors]],  # Detect(P3, P4, P5)
  ]

In [ ]:
# Import PyYAML module
import yaml

# Define the YAML content as a dictionary
data = {'train': '/content/Fire_Detector/Dataset/train/images',
        'val': '/content/Fire_Detector/Dataset/train/images',
        'nc': 2,
        'names': ['smoke', 'fire']} #ar trebui sa detecteze atat focul, cat si fumul

# Set the file path and name
file_path = '/content/data.yaml'

# Open the file in write mode and write the YAML content
with open(file_path, 'w') as file:
    documents = yaml.dump(data, file)

In [ ]:
#convert our pascal voc to yolov5, because the dataset is not compatible by the default, I had a mental breakdown here
!pip install pylabel


In [ ]:
from pylabel import importer
dataset = importer.ImportVOC(path="/content/Fire_Detector/Dataset/train/labels")
dataset.export.ExportToYoloV5(output_path="/content/Fire_Detector/Dataset/train/labels") 

In [ ]:
dataset = importer.ImportVOC(path="/content/Fire_Detector/Dataset/validation/labels")
dataset.export.ExportToYoloV5(output_path="/content/Fire_Detector/Dataset/validation/labels") 

# **Train on new dataset**

In [ ]:
# train yolov5s on custom data for 10 epochs(10 generatii), pentru acuratete mai mare, putem pune mai multe
# time its performance
%%time
%cd /content/yolov5/
!python train.py --img 416 --batch 16 --epochs 10 --data '../data.yaml' --cfg ./models/custom_yolov5s.yaml --weights '' --name yolov5s_results  --cache

# **Test on Images and Videos**

# New Section

In [ ]:
!python detect.py --weights /content/yolov5/runs/train/yolov5s_results9/weights/best.pt --img 416 --conf 0.4 --source /content/Fire_Detector/Dataset/validation/images

In [ ]:
import glob
from IPython.display import Image, display

for imageName in glob.glob('/content/yolov5/inference/output/*.jpg'):
    display(Image(filename=imageName))
    print("\n")